<a href="https://colab.research.google.com/github/TrinhHoangKhang/AIO-2024/blob/main/Homework/Module3/Week4/IMDB_film_review_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 9.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
import numpy as np
import string
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import contractions
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [3]:
import os
from google.colab import userdata
os.environ["KAGGLE_KEY"]  = userdata.get("KAGLE_TOKEN")
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGLE_USERNAME')
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews
License(s): other
100% 25.7M/25.7M [00:02<00:00, 20.1MB/s]
100% 25.7M/25.7M [00:02<00:00, 10.4MB/s]


In [4]:
!unzip '/content/imdb-dataset-of-50k-movie-reviews.zip'

Archive:  /content/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [5]:
df = pd.read_csv('/content/IMDB Dataset.csv')
df.info()
print(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB
                                              review sentiment
0  One of the other reviewers has mentioned that ...  positive
1  A wonderful little production. <br /><br />The...  positive
2  I thought this was a wonderful way to spend ti...  positive
3  Basically there's a family where a little boy ...  negative
4  Petter Mattei's "Love in the Time of Money" is...  positive
5  Probably my all-time favorite movie, a story o...  positive
6  I sure would like to see a resurrection of a u...  positive
7  This show was an amazing, fresh & innovative i...  negative
8  Encouraged by the positive comments about this...  negative
9  If you like original gut wrenching laughter yo...  positive


#### Preprocessing

In [12]:
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()
def preprocess(text):
  # Lowercase
  text = text.lower()

  # Remove punctuation
  text = text.translate(str.maketrans('', '', string.punctuation))

  # Remove html tag
  soup = BeautifulSoup(text)
  text = soup.get_text()

  # Tokenize
  text_tokens = text.split()

  # Remove stopword
  text_tokens = [token for token in text_tokens if token not in stop_words]

  # Stemming
  text_tokens = [stemmer.stem(token) for token in text_tokens]

  # Reconnect tokens into text (for sklearn)
  text = ' '.join(text_tokens)

  return text

In [16]:
# Vectorize + Get the X for training
reviews = df['review'].values
reviews = np.array([preprocess(review) for review in reviews])
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(reviews)
print(f"Shape of X: {X.shape}")
print(f"First row: {X[0].toarray()}")

Shape of X: (50000, 142260)
First row: [[0 0 0 ... 0 0 0]]


#### Prepare X and y for traning

In [17]:
# We have X already
# Get the y
sentiments = df['sentiment'].values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(sentiments)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#### Train the model

In [19]:
random_forest = RandomForestClassifier()
ada_boost = AdaBoostClassifier()

random_forest.fit(X_train, y_train)
ada_boost.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier()

#### Evaluation

In [20]:
# Accuracy score
rf_predict = random_forest.predict(X_test)
ada_predict = ada_boost.predict(X_test)

rf_accuracy = accuracy_score(y_test, rf_predict)
ada_accuracy = accuracy_score(y_test, ada_predict)
print(f"Random forest: {rf_accuracy}")
print(f"Ada boost: {ada_accuracy}")

Random forest: 0.8542
Ada boost: 0.8041
